In [15]:
from langchain_ollama import OllamaLLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import DirectoryLoader, TextLoader
import os

In [16]:
file_path = r"C:\Users\r.esmkhani\Desktop\langChain\rezone.docx"

# Verify file exists
if not os.path.exists(file_path):
    print(f"Error: The file '{file_path}' does not exist. Please check the path and try again.")
else:
    print(f"File '{file_path}' found successfully.")

File 'C:\Users\r.esmkhani\Desktop\langChain\rezone.docx' found successfully.


In [17]:
try:
    loader = Docx2txtLoader(file_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} document(s).")
    if documents:
        print("First 100 characters of the document:")
        print(documents[0].page_content[:100])
except Exception as e:
    print(f"Error loading the file: {e}")

Loaded 1 document(s).
First 100 characters of the document:
صفحه: 1

آدمیان
آد آرویا قلی پو
۱ ۱


صفحه: 2

سرشناسه: قلی‌پورء زویاء ۱۳۶۰
عنوان و نام‌پدیدآور: آدم


In [18]:
try:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Created {len(chunks)} chunks.")
    if chunks:
        print("First 100 characters of the first chunk:")
        print(chunks[0].page_content[:100])
except Exception as e:
    print(f"Error chunking the document: {e}")

Created 193 chunks.
First 100 characters of the first chunk:
صفحه: 1

آدمیان
آد آرویا قلی پو
۱ ۱


صفحه: 2

سرشناسه: قلی‌پورء زویاء ۱۳۶۰
عنوان و نام‌پدیدآور: آدم


In [19]:
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

C:\Users\r.esmkhani\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
vector_store = FAISS.from_documents(chunks, embedding_model)
print("Vector store created.")

Vector store created.


In [21]:
llm = OllamaLLM(model="gemma3")

In [22]:
prompt_template = """با استفاده از اطلاعات زیر، به سوال پاسخ دهید. اگر نمی‌دانید، بگویید که نمی‌دانم.

زمینه: {context}

سوال: {question}

پاسخ: """
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [23]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": prompt}
)

In [28]:
query = "مادر برای اینکه پسر از رعد و برق نترسد آن را به چه چیزی تشبیه کرد ؟"
result = rag_chain({"query": query})
print(result["result"])

مادر برای اینکه پسر از رعد و برق نترسد، آن را به یک "هوا" با نورهای جالب تشبیه کرد که شبیه ریشه درخت است.
